# Data Masking Sample 
### Using StackOverflow2010 Database from BrentOzar.com

My task here is to identify the sensitivity of the database, so that I can capture that in SQL Data Catalog (my system of record for data classification), then use that metadata to define my masking operations before secondary use of that database.

In [1]:
-- Get a list of tables and views in the current database
SELECT table_catalog [database], table_schema [schema], table_name [name], table_type [type]
FROM [StackOverflow2010].INFORMATION_SCHEMA.TABLES;

(9 rows affected)

Total execution time: 00:00:01.133

database,schema,name,type
StackOverflow2010,dbo,Comments,BASE TABLE
StackOverflow2010,dbo,LinkTypes,BASE TABLE
StackOverflow2010,dbo,PostLinks,BASE TABLE
StackOverflow2010,dbo,Posts,BASE TABLE
StackOverflow2010,dbo,PostTypes,BASE TABLE
StackOverflow2010,dbo,Users,BASE TABLE
StackOverflow2010,dbo,Votes,BASE TABLE
StackOverflow2010,dbo,VoteTypes,BASE TABLE
StackOverflow2010,dbo,Badges,BASE TABLE


First, let's look at the Users table before masking. Some recognisable names here. Location is pretty specific in some cases, and AboutMe is sometimes very identifiable. WebsiteUrl definitely so. EmailHash is null in all cases. What if it started to be used though? This is something I would flag up for refactoring; if it's not used, let's not leave it in the schema. Sometimes there's some common-sense risk reduction that we can use this opportunity to implement.

In [8]:
SELECT Top(10) * FROM [StackOverflow2010].dbo.Users;

(10 rows affected)

Total execution time: 00:00:00.114

Id,AboutMe,Age,CreationDate,DisplayName,DownVotes,EmailHash,LastAccessDate,Location,Reputation,UpVotes,Views,WebsiteUrl,AccountId
-1,"<p>Hi, I'm not really a person.</p> <p>I'm a background process that helps keep this site clean!</p> <p>I do things like</p> <ul> <li>Randomly poke old unanswered questions every hour so they get some attention</li> <li>Own community questions and answers so nobody gets unnecessary reputation from them</li> <li>Own downvotes on spam/evil posts that get permanently deleted</li> <li>Own suggested edits from anonymous users</li> <li><a href=""http://meta.stackexchange.com/a/92006"">Remove abandoned questions</a></li> </ul>",NULL,2008-07-31 00:00:00.000,Community,980920,NULL,2008-08-26 00:16:53.810,on the server farm,1,274835,649,http://meta.stackexchange.com/,-1
1,"<p><a href=""http://www.codinghorror.com/blog/archives/001169.html"" rel=""nofollow"">Stack Overflow Valued Associate #00001</a></p> <p>Wondering how our software development process works? <a href=""http://www.youtube.com/watch?v=08xQLGWTSag"" rel=""nofollow"">Take a look!</a></p> <p>Find me <a href=""http://twitter.com/codinghorror"" rel=""nofollow"">on twitter</a>, or <a href=""http://www.codinghorror.com/blog"" rel=""nofollow"">read my blog</a>. Don't say I didn't warn you <em>because I totally did</em>.</p> <p>However, <a href=""http://www.codinghorror.com/blog/2012/02/farewell-stack-exchange.html"" rel=""nofollow"">I no longer work at Stack Exchange, Inc</a>. I'll miss you all. Well, <em>some</em> of you, anyway. :)</p>",NULL,2008-07-31 14:22:31.287,Jeff Atwood,1309,NULL,2018-08-29 02:34:22.893,"El Cerrito, CA",44300,3367,408587,http://www.codinghorror.com/blog/,1
2,"<p>Developer on the Stack Overflow team. Find me on</p> <p><a href=""http://www.twitter.com/SuperDalgas"" rel=""nofollow noreferrer"">Twitter</a> <br><br> <a href=""http://blog.stackoverflow.com/2009/05/welcome-stack-overflow-valued-associate-00003/"">Stack Overflow Valued Associate #00003</a></p>",NULL,2008-07-31 14:22:31.287,Geoff Dalgas,88,NULL,2018-08-23 17:31:56.427,"Corvallis, OR",3491,650,23966,http://stackoverflow.com,2
3,"<p><a href=""http://blog.stackoverflow.com/2009/01/welcome-stack-overflow-valued-associate-00002/"">Developer on the Stack Overflow team</a>.</p> <p>Was dubbed <strong>SALTY SAILOR</strong> by Jeff Atwood, as filth and flarn would oft-times fly when dealing with a particularly nasty bug!</p> <ul> <li>Twitter me: <a href=""http://twitter.com/jarrod_dixon"" rel=""nofollow noreferrer"">jarrod_dixon</a></li> <li>Email me: jarrod.m.dixon@gmail.com</li> </ul>",NULL,2008-07-31 14:22:31.287,Jarrod Dixon,100,NULL,2018-08-30 20:56:23.897,"Raleigh, NC, United States",13418,7285,24396,http://jarroddixon.com,3
4,"<p>I am:</p> <ul> <li>the co-founder and CEO of <a href=""http://stackoverflow.com"">Stack Overflow</a></li> <li>the co-founder of <a href=""http://www.fogcreek.com"" rel=""nofollow noreferrer"">Fog Creek Software</a></li> <li>the creator of <a href=""http://trello.com"" rel=""nofollow noreferrer"">Trello</a> (now owned by Atlassian)</li> </ul> <p>You can find me on my rarely-updated blog, <a href=""http://joelonsoftware.com"" rel=""nofollow noreferrer"">Joel on Software</a>.</p>",NULL,2008-07-31 14:22:31.317,Joel Spolsky,96,NULL,2018-08-14 22:18:15.227,"New York, NY",28768,797,73755,http://www.joelonsoftware.com/,4
5,"<p>Technical Evangelist at Microsoft, specializing in ASP.NET MVC.</p> <p>I don't use this site anymore because the moderators close or delete far too many of the useful questions.</p>",NULL,2008-07-31 14:22:31.317,Jon Galloway,34,NULL,2018-08-29 16:48:35.993,"San Diego, CA",39172,781,11700,http://weblogs.asp.net/jgalloway/,5
8,"<p>This is a puppet test account I use to validate ""regular user"" stuff on the site</p> <p>-- <a href=""http://stackoverflow.com/users/1/jeff-atwood"" rel=""nofollow"">Jeff Atwood</a>",NULL,2008-07-31 21:33:24.057,Eggs McLaren,9,NULL,2018-04-09 02:04:55.577,,942,12,6372,,6
9,<p>Independent software engine

Next, the Badges table.

In [4]:
SELECT Top(5) * FROM [StackOverflow for Masking].dbo.Badges;

(5 rows affected)

Total execution time: 00:00:00.0859050

Id,Name,UserId,Date
82946,Teacher,3718,2008-09-15 08:55:03.923
82947,Teacher,994,2008-09-15 08:55:03.957
82949,Teacher,3893,2008-09-15 08:55:03.957
82950,Teacher,4591,2008-09-15 08:55:03.957
82951,Teacher,5196,2008-09-15 08:55:03.957


Slightly odd normalisation decision perhaps, but not sensitive.
OK, leave that one, Votes next.

In [5]:
SELECT Top(5) * FROM [StackOverflow for Masking].dbo.Votes;

(5 rows affected)

Total execution time: 00:00:00.1026600

Id,PostId,UserId,BountyAmount,VoteTypeId,CreationDate
1,1,NULL,NULL,2,2008-07-31 00:00:00.000
2,3,NULL,NULL,2,2008-07-31 00:00:00.000
3,2,NULL,NULL,2,2008-07-31 00:00:00.000
4,4,NULL,NULL,2,2008-07-31 00:00:00.000
5,6,NULL,NULL,2,2008-07-31 00:00:00.000


Can't see anything there. If we've masked the users well, what they've voted for won't be very interesting. Or _will_ it? If I know the context, I might be able to do reidentification. Jon Skeet is famously prolific for example. One of those slightly tricky calls that requires business context and a risk assessment. Who is going to see the masked database? What would happen id they reidentified using their knowledge?

In [6]:
SELECT Top(5) * FROM [StackOverflow for Masking].dbo.Comments;

(5 rows affected)

Total execution time: 00:00:00.1368510

Id,CreationDate,PostId,Score,Text,UserId
1,2008-09-06 08:07:10.730,35314,36,not sure why this is getting downvoted -- it is correct! Double check it in your compiler if you don't believe him!,1
2,2008-09-06 08:09:52.330,35314,8,"Yeah, I didn't believe it until I created a console app - but good lord! Why would they give you the rope to hang yourself! I hated that about VB.NET - the OrElse and AndAlso keywords!",3
4,2008-09-06 08:42:16.980,35195,0,"I don't see an accepted answer now, I wonder how that got unaccepted. Incidentally, I would have marked an accepted answer based on the answers available at the time. Also, accepted doesn't mean Best :)",380
9,2008-09-06 12:26:30.060,47239,0,"Jonathan: Wow! Thank you for all of that, you did an amazing amount of work!",4550
10,2008-09-06 13:38:23.647,45651,6,It will help if you give some details of which database you are using as techniques vary.,242


Going to need to handle that text I think. Ok, Posts next.

In [3]:
SELECT Top(5) * FROM [StackOverflow2010].dbo.Posts;

(5 rows affected)

Total execution time: 00:00:01.699

Id,AcceptedAnswerId,AnswerCount,Body,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,LastActivityDate,LastEditDate,LastEditorDisplayName,LastEditorUserId,OwnerUserId,ParentId,PostTypeId,Score,Tags,Title,ViewCount
4,7,13,"<p>I want to use a track-bar to change a form's opacity.</p> <p>This is my code:</p> <pre><code>decimal trans = trackBar1.Value / 5000; this.Opacity = trans; </code></pre> <p>When I build the application, it gives the following error:</p> <blockquote> <p>Cannot implicitly convert type <code>'decimal'</code> to <code>'double'</code>.</p> </blockquote> <p>I tried using <code>trans</code> and <code>double</code> but then the control doesn't work. This code worked fine in a past VB.NET project.</p>",NULL,1,2012-10-31 16:42:47.213,2008-07-31 21:42:52.667,41,2018-07-02 17:55:27.247,2018-07-02 17:55:27.247,Rich B,6786713,8,0,1,573,<c#><floating-point><type-conversion><double><decimal>,Convert Decimal to Double?,37080
6,31,5,"<p>I have an absolutely positioned <code>div</code> containing several children, one of which is a relatively positioned <code>div</code>. When I use a <strong>percentage-based width</strong> on the child <code>div</code>, it collapses to '0' width on <a href=""http://en.wikipedia.org/wiki/Internet_Explorer_7"" rel=""noreferrer"">Internet&nbsp;Explorer&nbsp;7</a>, but not on Firefox or Safari.</p> <p>If I use <strong>pixel width</strong>, it works. If the parent is relatively positioned, the percentage width on the child works.</p> <ol> <li>Is there something I'm missing here?</li> <li>Is there an easy fix for this besides the <em>pixel-based width</em> on the child?</li> <li>Is there an area of the CSS specification that covers this?</li> </ol>",NULL,0,NULL,2008-07-31 22:08:08.620,10,2016-03-19 06:10:52.170,2016-03-19 06:05:48.487,Rich B,63550,9,0,1,256,<html><css><css3><internet-explorer-7>,Percentage width child element in absolutely positioned parent on Internet Explorer 7,16306
7,0,0,<p>An explicit cast to double like this isn't necessary:</p> <pre><code>double trans = (double) trackBar1.Value / 5000.0; </code></pre> <p>Identifying the constant as <code>5000.0</code> (or as <code>5000d</code>) is sufficient:</p> <pre><code>double trans = trackBar1.Value / 5000.0; double trans = trackBar1.Value / 5000d; </code></pre>,NULL,0,NULL,2008-07-31 22:17:57.883,0,2017-12-16 05:06:57.613,2017-12-16 05:06:57.613,NULL,4020527,9,4,2,401,NULL,NULL,0
9,1404,64,"<p>Given a <code>DateTime</code> representing a person's birthday, how do I calculate their age in years? </p>",NULL,7,2011-08-16 19:40:43.080,2008-07-31 23:40:59.743,399,2018-07-25 11:57:14.110,2018-04-21 17:48:14.477,Rich B,3956566,1,0,1,1743,<c#><.net><datetime>,How do I calculate someone's age in C#?,480476
11,1248,35,"<p>Given a specific <code>DateTime</code> value, how do I display relative time, like:</p> <ul> <li>2 hours ago</li> <li>3 days ago</li> <li>a month ago</li> </ul>",NULL,3,2009-09-04 13:15:59.820,2008-07-31 23:55:37.967,529,2018-07-05 04:00:56.633,2017-06-04 15:51:19.780,user2370523,6479704,1,0,1,1348,<c#><datetime><time><datediff><relative-time-span>,Calculate relative time in C#,136033


LastEditorDisplayName, going on this list.
How about Body? This is a bit trickier; the body is in most cases a public post on a website about a technical matter. How could that be confidential?
But here's the rub; GDPR isn't necessarily about confidentiality. It's about taking a risk-based approach to the handling of personal data. The first thing we have to identify is whether it is personal. 
```
Personal data is information that relates to an identified or identifiable individual.

What identifies an individual could be as simple as a name or a number or could include other identifiers such as an IP address or a cookie identifier, or other factors.
```
https://ico.org.uk/for-organisations/guide-to-data-protection/guide-to-the-general-data-protection-regulation-gdpr/key-definitions/what-is-personal-data/

In [5]:
SELECT Top(10) * FROM [StackOverflow2010].dbo.Posts where Body like '%@%';

(10 rows affected)

Total execution time: 00:00:00.517

Id AcceptedAnswerId AnswerCount Body ClosedDate CommentCount CommunityOwnedDate CreationDate FavoriteCount LastActivityDate LastEditDate LastEditorDisplayName LastEditorUserId OwnerUserId ParentId PostTypeId Score Tags Title ViewCount 18 0 0 <p>For a table like this:</p>

<pre><code>CREATE TABLE binary_data (
 id INT(4) NOT NULL AUTO_INCREMENT PRIMARY KEY,
 description CHAR(50),
 bin_data LONGBLOB,
 filename CHAR(50),
 filesize CHAR(50),
 filetype CHAR(50)
);
</code></pre>

<p>Here is a PHP example:</p>

<pre><code>&lt;?php
 // store.php3 - by Florian Dittmer &lt;dittmer@gmx.net&gt;
 // Example php script to demonstrate the storing of binary files into
 // an sql database. More information can be found at http://www.phpbuilder.com/
?&gt;

&lt;html&gt;
 &lt;head&gt;&lt;title&gt;Store binary data into SQL Database&lt;/title&gt;&lt;/head&gt;

 &lt;body&gt;
 &lt;?php
 // Code that will be executed if the form has been submitted:

 if ($submit) {
 // Connect to the database (you may have to adjust
 // the hostname, username or password).

 mysql_connect("localhost", "root", "password");
 mysql_select_db("binary_data");

 $data = mysql_real_escape_string(fread(fopen($form_data, "r"), filesize($form_data)));

 $result = mysql_query("INSERT INTO binary_data (description, bin_data, filename, filesize, filetype) ".
 "VALUES ('$form_description', '$data', '$form_data_name', '$form_data_size', '$form_data_type')");

 $id= mysql_insert_id();
 print "&lt;p&gt;This file has the following Database ID: &lt;b&gt;$id&lt;/b&gt;";

 mysql_close();
 } else {

 // else show the form to submit new data:
 ?&gt;
 &lt;form method="post" action="&lt;?php echo $PHP_SELF; ?&gt;" enctype="multipart/form-data"&gt;
 File Description:&lt;br&gt;
 &lt;input type="text" name="form_description" size="40"&gt;
 &lt;input type="hidden" name="MAX_FILE_SIZE" value="1000000"&gt;
 &lt;br&gt;File to upload/store in database:&lt;br&gt;
 &lt;input type="file" name="form_data" size="40"&gt;
 &lt;p&gt;&lt;input type="submit" name="submit" value="submit"&gt;
 &lt;/form&gt;

 &lt;?php
 }
 ?&gt;
 &lt;/body&gt;
&lt;/html&gt;
</code></pre>
 NULL 2 NULL 2008-08-01 05:12:44.193 0 2016-06-02 05:56:26.060 2016-06-02 05:56:26.060 Jeff Atwood 126039 0 17 2 56 NULL NULL 0 27 0 0 <p>@jeff</p>

<p>IMHO yours seems a little long. However it does seem a little more robust with support for "yesterday" and "years". But in my experience when this is used the person is most likely to view the content in the first 30 days. It is only the really hardcore people that come after that. So that is why I usually elect to keep this short and simple.</p>

<p>This is the method I am currently using on one of my websites. This only returns a relative day, hour, time. And then the user has to slap on "ago" in the output.</p>

<pre><code>public static string ToLongString(this TimeSpan time)
{
 string output = String.Empty;

 if (time.Days &gt; 0)
 output += time.Days + " days ";

 if ((time.Days == 0 || time.Days == 1) &amp;&amp; time.Hours &gt; 0)
 output += time.Hours + " hr ";

 if (time.Days == 0 &amp;&amp; time.Minutes &gt; 0)
 output += time.Minutes + " min ";

 if (output.Length == 0)
 output += time.Seconds + " sec";

 return output.Trim();
}
</code></pre>
 NULL 0 2009-09-04 13:15:59.820 2008-08-01 12:17:19.357 0 2008-08-01 13:16:49.127 2008-08-01 13:16:49.127 Nick Berardi 17 17 11 2 29 NULL NULL 0 73 0 0 <p>@Jax: The <code>extern "C"</code> thing matters, very very much. If a header file doesn't have one, then (unless it's a C++-only header file), you would have to enclose your <code>#include</code> with it:</p>

<pre><code>extern "C" {
#include &lt;sys/socket.h&gt;
// include other similarly non-compliant header files
}
</code></pre>

<p>Basically, anytime where a C++ program wants to link to C-based facilities, the <code>extern "C"</code> is vital. In practical terms, it means that the names used in external references will not be mangled, like normal C++ names would. <a href="http://www.parashift

There are references to other people, that looks like someone's personal blog url there as well. This is identifiable data. It might be low in risk and confidentiality, but that's an assessment for later.